In [100]:
import tensorflow as tf

In [101]:
class Darknet_BN_Leaky(tf.keras.Model):
    def __init__(self, 
                 filter, 
                 kernel, 
                 stride,
                 type_of_network, 
                 padding = 'same'):
        if type_of_network == '1d':
            self.conv = tf.keras.layers.Conv1D(filter,
                                               kernel,
                                               strides = stride,
                                               padding=padding)
        if type_of_network == '2d':
            self.conv = tf.keras.layers.Conv2D(filter,
                                               kernel,
                                               strides = stride,
                                               padding=padding)
                                               
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.lrelu = tf.keras.layers.LeakyReLU()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.batch_norm(x, training=training)
        x = self.lrelu(x)
        return x

In [102]:
class Res_unit(tf.keras.Model):
    def __init__(self, 
                 kernels, 
                 filters,
                 strides, 
                 type_of_network):

        filter1, filter2 = filters
        kernel1, kernel2 = kernels
        stride1, stride2 = strides
        

        self.conv1_1 = Darknet_BN_Leaky(filter1,
                                        kernel1,
                                        strides=stride1,
                                        type_of_network = type_of_network
                                        )

        self.conv1_2 = Darknet_BN_Leaky(filter2,
                                        kernel2,
                                        strides=stride2,
                                        type_of_network = type_of_network
                                        )
        

    def call(self, input_tensor):
        
        x = self.conv1_1(input_tensor)
        x = self.conv1_2(x)

        x += input_tensor

        return x 

In [103]:
class ResBlock_N(tf.keras.Model):

    def __init__(self, 
                 num, 
                 filters, 
                 kernels, 
                 strides,
                 type_of_network, 
                 padding = 'same'):
        if type_of_network == '1d':
            self.DBL = tf.keras.layers.Conv1D(filters[0],
                                              kernels[0],
                                              strides = strides[0],
                                              padding=padding[0])
        if type_of_network == '2d':
            self.DBL = tf.keras.layers.Conv2D(filters[0],
                                              kernels[0],
                                              strides = strides[0],
                                              padding=padding[0])
        self.res_body = []
        for ind in range(1, num):
            self.res_body.append(Res_unit(filters[ind],
                                        kernels[ind],
                                        strides=strides[ind],
                                        type_of_network = type_of_network
                                        ))

    def call(self, input_tensor):
        x = self.DBL(input_tensor)

        for Res_Unit in self.res_body:
            x = Res_Unit(x)

        return x

In [1]:
from tensorflow.keras.layers import Input

def build_model(image_height, image_width):

    x = Input(shape = (image_height, image_width), name = 'Input')

    x = Darknet_BN_Leaky(32, 3, 1, '1d')